In [72]:
import requests
from pyquery import PyQuery as pq
import csv
from collections import Counter
import Levenshtein

In [151]:
def clean_name(x):
    return x.replace('-',' ')\
            .replace("'",'')\
            .replace('יי','י')\
            .replace('*','')\
            .replace('"','')\
            .split('  ')[0].strip()


In [167]:
def get_cities(html, all_addresses):
    html = pq(html)
    addresses = html.find('[itemprop="streetAddress"]')
    names = html.find('[itemprop="name"]')
    addresses = [pq(x).text()+','+pq(y).text() for x,y in zip(names, addresses)]
#     print(len(addresses), '\n'.join(addresses))
    addresses = [x for x in addresses if x not in all_addresses]
    all_addresses.extend(addresses)
    return [clean_name(x.split(',')[-1]) for x in addresses]

In [168]:
def get_cities_for_query(query):
    basic_url = 'http://www.d.co.il/SearchResults'
    more_url = 'http://www.d.co.il/DOTNET/ajax/GetMoreResults'
    
    cities = []
    all_addresses = []

    s = requests.session()
    parameters={
        'query':query, 
        'type':'tag', 
        'location': ''
    }
    resp = s.get(basic_url, params=parameters)
    print(resp.url)
    page = resp.text
    cities.extend(get_cities(page, all_addresses))
#     print(1, len(cities))

    page_number = 2
    while True:
        parameters.update({
            'type': 'tag',
            'pageNumber': page_number,
            'sortType': 3,
            'findOnlyInCity': 0,
            'locationId': 0
        })
        response = s.post(more_url, data=parameters).json()
        if not response.get('success'):
            break
        html = response['html']
        cities.extend(get_cities(html, all_addresses))
        page_number += 1
#         print(page_number, len(cities))
    
    return cities

In [169]:
munis = csv.DictReader(open('munis.csv'))
muni_to_city = csv.DictReader(open('municipality-to-city.csv'))
idx_col = 'index_peripheral_2004_cluster_from_1_to_10_1_compact_most'
munis = dict(
    (clean_name(x['name_municipality']),
     dict(pop=float(x['population']),
          idx=float(x[idx_col]) if x[idx_col] else None
         ))
    for x in munis
)
muni_to_city = dict(
    (clean_name(x['city']), clean_name(x['municipality']))
    for x in muni_to_city
    if x['city'].strip() != ''
)

In [170]:
def match(x,y,level=0):
    ret = (level >= 0) and (x==y)
    ret = ret or (level >= 1) and (x in y or y in x)
    ret = ret or (level >= 2) and (Levenshtein.distance(x, y) < level)
    return ret

In [181]:
queries = [
    'חנויות ספרים',
    'מספרות ועיצוב שיער',
    'ציפורניים - בנייה וטיפוח',    
]
allres = {}
idxres = {}
for muni, values in munis.items():
    allres[muni] = dict(
        name=muni,
        idx=values['idx'],
        pop=values['pop']
    )
    idxres.setdefault(values['idx'], {})
    idxrec = idxres[values['idx']]
    idxrec.setdefault('idx', values['idx'])
    idxrec.setdefault('pop', 0)
    idxrec['pop'] += values['pop']
    for q in queries:
        allres[muni][q] = 0
        idxrec[q] = 0
for q in queries:
    cities = get_cities_for_query(q)
    print(len(cities))
    cities = dict(Counter(cities))
    for city, count in cities.items():
        found = False
        converted = False
        while not found:
            for level in range(4):
                for muni, values in munis.items():
                    if match(muni, city, level):
                        found = True
                        allres[muni][q] += count
                        idxres[values['idx']][q] += count
                        break
                if found:
                    break
            if found or converted:
                break
            for level in range(4):
                for k, v in muni_to_city.items():
                    if match(k, city, level):
                        found = True
                        city = v
                        converted = True
                        break
                if found:
                    break
            if not found:            
                print('MISSING',city)
                break


http://www.d.co.il/SearchResults?query=%D7%97%D7%A0%D7%95%D7%99%D7%95%D7%AA+%D7%A1%D7%A4%D7%A8%D7%99%D7%9D&location=&type=tag
591
MISSING המרכז האקדמי רופין
MISSING אירפורט סיטי
http://www.d.co.il/SearchResults?query=%D7%9E%D7%A1%D7%A4%D7%A8%D7%95%D7%AA+%D7%95%D7%A2%D7%99%D7%A6%D7%95%D7%91+%D7%A9%D7%99%D7%A2%D7%A8&location=&type=tag
4912
MISSING רמת אפעל
MISSING אירפורט סיטי
MISSING תל חי
http://www.d.co.il/SearchResults?query=%D7%A6%D7%99%D7%A4%D7%95%D7%A8%D7%A0%D7%99%D7%99%D7%9D+-+%D7%91%D7%A0%D7%99%D7%99%D7%94+%D7%95%D7%98%D7%99%D7%A4%D7%95%D7%97&location=&type=tag
1486


In [182]:
kk=list(allres.keys())[:15]
for k in kk:
    print(k,allres[k])
kk=list(idxres.keys())[:15]
for k in kk:
    print(k,idxres[k])


מזרעה {'pop': 3700.0, 'ציפורניים - בנייה וטיפוח': 0, 'name': 'מזרעה', 'חנויות ספרים': 0, 'idx': 4.0, 'מספרות ועיצוב שיער': 2}
הגלבוע {'pop': 28900.0, 'ציפורניים - בנייה וטיפוח': 0, 'name': 'הגלבוע', 'חנויות ספרים': 0, 'idx': 4.0, 'מספרות ועיצוב שיער': 0}
מיתר {'pop': 7300.0, 'ציפורניים - בנייה וטיפוח': 3, 'name': 'מיתר', 'חנויות ספרים': 2, 'idx': 4.0, 'מספרות ועיצוב שיער': 3}
באקה אל גרביה {'pop': 28100.0, 'ציפורניים - בנייה וטיפוח': 1, 'name': 'באקה אל גרביה', 'חנויות ספרים': 0, 'idx': None, 'מספרות ועיצוב שיער': 7}
כוכב יאיר {'pop': 9000.0, 'ציפורניים - בנייה וטיפוח': 1, 'name': 'כוכב יאיר', 'חנויות ספרים': 2, 'idx': 7.0, 'מספרות ועיצוב שיער': 3}
פקיעין (בוקיעה) {'pop': 5700.0, 'ציפורניים - בנייה וטיפוח': 0, 'name': 'פקיעין (בוקיעה)', 'חנויות ספרים': 0, 'idx': 4.0, 'מספרות ועיצוב שיער': 1}
זבולון {'pop': 12700.0, 'ציפורניים - בנייה וטיפוח': 0, 'name': 'זבולון', 'חנויות ספרים': 0, 'idx': 5.0, 'מספרות ועיצוב שיער': 0}
חוף הכרמל {'pop': 29300.0, 'ציפורניים - בנייה וטיפוח': 0, 'name': 'ח

In [186]:
w=csv.DictWriter(open('allres.csv','w'), ['name', 'idx', 'pop']+queries)
w.writeheader()
w.writerows(list(allres.values()))

w=csv.DictWriter(open('idxres.csv','w'), ['idx', 'pop']+queries)
w.writeheader()
w.writerows(list(idxres.values()))

del w
